In [44]:
import mwxml
import re
from tqdm import tqdm

dump = mwxml.Dump.from_file(open("../data/dewiki-20220520-pages-articles-multistream1.xml"))
print(dump.site_info.name, dump.site_info.dbname)

Wikipedia dewiki


In [48]:
pattern = re.compile('.*\[\[Kategorie:(.*)\]\].*')
lines = []
with open("../data/title-categories-map.txt", mode="w") as outfile:
    for i, page in enumerate(tqdm(dump)):
        line = ""
        line += page.title + ","
        for revision in page:
            for match in pattern.finditer(revision.text):
                line += match.group(1) + ","
        line = line.strip(",")
        #lines.append(line)
        outfile.write(line+"\n")

16380it [03:18, 82.60it/s] 


KeyboardInterrupt: 